In [5]:
## Cleaning the data
import pandas as pd
def clean(data: pd.DataFrame, pmc: pd.DataFrame) -> pd.DataFrame:

    # Base da Dados Internos
    data = data.drop(columns=['Mês/ano', 'CONCATENAR', '% Ating. Meta'])
    data = data.rename(columns={'Ano':'ano'})
    data.at[36839, 'Setor'] = 'COMERC.P/ AUTOMOTIVO'
    data.at[45332, 'Setor'] = 'LOJA VAREJISTA/ATAC'
    data =  data.sort_values(by=['ano', 'Mês'])
    data = data.fillna(method='ffill')
    data['Desc %'] = data['Desc %'].str.replace('-','')
    data.drop_duplicates(inplace=True)
    data['Vl Liquido Final'] = data['Vl Liquido Final'].str.replace('.', '')
    data['Vl Liquido Final'] = data['Vl Liquido Final'].str.replace(',', '.')
    data['Vl Bruto'] = data['Vl Bruto'].str.replace('.', '')
    data['Vl Bruto'] = data['Vl Bruto'].str.replace(',', '.')
    data['Desconto R$'] = data['Desconto R$'].str.replace('.', '')
    data['Desconto R$'] = data['Desconto R$'].str.replace(',', '.')
    data['VL Tabela'] = data['VL Tabela'].str.replace('.', '')
    data['VL Tabela'] = data['VL Tabela'].str.replace(',', '.')
    data['IPCA ES'] = data['IPCA ES'].str.replace(',', '.')
    data['IPCA BR'] = data['IPCA BR'].str.replace(',', '.')
    data['Taxa Ac. TRI % PIB'] = data['Taxa Ac. TRI % PIB'].str.replace(',', '.')
    data['Desc %'] = data['Desc %'].str.replace('%', '')
    data['Vl Liquido Final'] = pd.to_numeric(data['Vl Liquido Final'])
    data['Vl Bruto'] = pd.to_numeric(data['Vl Bruto'])
    data['Desconto R$'] = pd.to_numeric(data['Desconto R$'])
    data['VL Tabela'] = pd.to_numeric(data['VL Tabela'])
    data['IPCA ES'] = pd.to_numeric(data['IPCA ES'])
    data['IPCA BR'] = pd.to_numeric(data['IPCA BR'])
    data['Taxa Ac. TRI % PIB'] = pd.to_numeric(data['Taxa Ac. TRI % PIB'])
    data['Desc %'] = pd.to_numeric(data['Desc %'])
    data['Desc %'] = data['Desc %']/100
    data['ano'] = pd.to_numeric(data['ano'])
    data['Mês'] = pd.to_numeric(data['Mês'])
    data['Dia'] = 1
    data_to_time = data[['ano', 'Mês', 'Dia']].rename(columns={'ano': 'year', 'Mês': 'month', 'Dia': 'day'})
    data['Data'] = pd.to_datetime(data_to_time)
    data.drop(columns=['ano', 'Mês', 'Dia'], inplace=True)
    data.sort_values(by=['Data'])
    data.drop(columns=['Segmento'], inplace=True)
    data = data.dropna(subset=['Setor'])
    data = data.dropna(subset=['Desc %'])
    data[data.duplicated()]
    data.drop_duplicates(inplace=True)

    # Dados Externos
    pmc = pmc[pmc['PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)'] != '...']
    meses_map = {
    'jan.': 'Jan',
    'fev.': 'Feb',
    'mar.': 'Mar',
    'abr.': 'Apr',
    'mai.': 'May',
    'jun.': 'Jun',
    'jul.': 'Jul',
    'ago.': 'Aug',
    'set.': 'Sep',
    'out.': 'Oct',
    'nov.': 'Nov',
    'dez.': 'Dec'
}
    # Converter a coluna de data
    pmc['Data'] = pd.to_datetime(pmc['Mês'].replace(meses_map, regex=True).str.replace(r'(\w+)-(\d+)', 
                                                       lambda m: f"01-{m.group(1)}-20{m.group(2)}",
                                                       regex=True),
                            format='%d-%b-%Y', errors='coerce')
    pmc['Data'] = pmc['Data'].dt.strftime('%Y-%m-%d')
    df_pmc_br = pmc[pmc['Brasil e Unidade da Federação'] == 'Brasil']
    df_pmc_es = pmc[pmc['Brasil e Unidade da Federação'] == 'Espírito Santo']
    df_pmc_es = df_pmc_es.drop(columns=['Brasil e Unidade da Federação','Tipos de índice', 'Mês'])
    df_pmc_br = df_pmc_br.drop(columns=['Brasil e Unidade da Federação','Tipos de índice', 'Data', 'Mês'])

    df_pmc_es.rename(columns={'PMC - Número-índice (2022=100) (Número-índice)': 'PMC - Número-índice (2022=100) (Número-índice)/ ES'}, inplace=True)
    df_pmc_es.rename(columns={'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)': 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES'}, inplace=True)
    df_pmc_es.rename(columns={'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)': 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES'}, inplace=True)
    df_pmc_br.rename(columns={'PMC - Número-índice (2022=100) (Número-índice)': 'PMC - Número-índice (2022=100) (Número-índice)/ BR'}, inplace=True)
    df_pmc_br.rename(columns={'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)': 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR'}, inplace=True)
    df_pmc_br.rename(columns={'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)': 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR'}, inplace=True)
    df_pmc_es = df_pmc_es.reset_index() 
    df_pmc_br = df_pmc_br.reset_index()
    df_combined = pd.concat([df_pmc_es, df_pmc_br], axis=1)
    df_combined = df_combined.drop(columns=['index'])

    df_combined = df_combined[['PMC - Número-índice (2022=100) (Número-índice)/ ES',
                            'PMC - Número-índice (2022=100) (Número-índice)/ BR',
                            'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES',
                            'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR',
                            'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES',
                            'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR',
                            'Data']]

    pmc = df_combined
    pmc['Data'] = pd.to_datetime(pmc['Data'])
    cleaned = pd.merge(data, pmc, on='Data', how='left')

    erase_comma_df = cleaned[['PMC - Número-índice (2022=100) (Número-índice)/ ES', 'PMC - Número-índice (2022=100) (Número-índice)/ BR', 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES', 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR', 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES', 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR']]

    for column in erase_comma_df.columns:
        cleaned[column] = cleaned[column].str.replace(',', '.')
        cleaned[column] = pd.to_numeric(cleaned[column], errors='coerce')


    return cleaned




In [6]:
import pandas as pd

def createDFbyUEN(originalData: pd.DataFrame, model: str) -> pd.DataFrame:
    df_analise = originalData.drop(['Desconto R$', 'Desc %'], axis=1)
    
    df_analise_model = df_analise[df_analise['UEN'] == model]
    
    columns_to_convert = [
        'VL Tabela', 'Vl Bruto', 'Vl Liquido Final', 'IPCA BR', 'IPCA ES',
        'Taxa Ac. TRI % PIB', 'PMC - Número-índice (2022=100) (Número-índice)/ ES',
        'PMC - Número-índice (2022=100) (Número-índice)/ BR',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR'
    ]
    
    for col in columns_to_convert:
        df_analise_model[col] = pd.to_numeric(df_analise_model[col], errors='coerce')
        df_analise_model.dropna(inplace=True)
    
    df_analise_corr_agg = df_analise_model.groupby(
        [pd.Grouper(key='Data', freq='ME'), 'Cliente', 'Setor', 'Veiculo']
    ).agg({
        'VL Tabela': 'sum',
        'Vl Bruto': 'sum',
        'Vl Liquido Final': 'sum',
        'IPCA BR': 'mean',
        'IPCA ES': 'mean',
        'Taxa Ac. TRI % PIB': 'mean',
        'PMC - Número-índice (2022=100) (Número-índice)/ ES': 'mean',
        'PMC - Número-índice (2022=100) (Número-índice)/ BR': 'mean',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES': 'mean',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR': 'mean',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES': 'mean',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR': 'mean'
    }).reset_index()
    

    df_analise_corr_agg = df_analise_corr_agg.groupby(pd.Grouper(key='Data', freq='M')).agg({
        'VL Tabela': 'sum',
        'Vl Bruto': 'sum',
        'Vl Liquido Final': 'sum',
        'IPCA BR': 'mean',
        'IPCA ES': 'mean',
        'Taxa Ac. TRI % PIB': 'mean',
        'PMC - Número-índice (2022=100) (Número-índice)/ ES': 'mean',
        'PMC - Número-índice (2022=100) (Número-índice)/ BR': 'mean',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES': 'mean',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR': 'mean',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES': 'mean',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR': 'mean'
    }).reset_index()
    
    return df_analise_corr_agg
    

In [7]:
def createDFbyOrigem(originalData: pd.DataFrame, model: str) -> pd.DataFrame:
    df = originalData
    df['AnoMes'] = df['Data'].dt.to_period('M')
    df_analise_origem = df.groupby(['AnoMes', 'Origem']).agg({
    'VL Tabela': 'sum',
    'Vl Bruto': 'sum',
    'Vl Liquido Final': 'sum',
    'Cliente': 'nunique',                 # Quantidade de clientes diferentes
    'Veiculo': 'nunique',                 # Quantidade de veículos diferentes
    'UEN': 'nunique',                     # Quantidade de UENs diferentes
    'Setor': 'nunique',                   # Quantidade de setores diferentes
    'IPCA BR': 'mean',
    'IPCA ES': 'mean',
    'Taxa Ac. TRI % PIB': 'mean',
    'PMC - Número-índice (2022=100) (Número-índice)/ ES': 'mean',
    'PMC - Número-índice (2022=100) (Número-índice)/ BR': 'mean',
    'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES': 'mean',
    'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR': 'mean',
    'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES': 'mean',
    'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR': 'mean',
}).reset_index()

    # Adicionar a contagem de contratos (ocorrências) para cada Origem
    df_analise_origem['Contratos'] = df.groupby(['AnoMes', 'Origem'])['Origem'].transform('count')

    # Renomear as colunas para deixar os nomes mais claros e evitar conflito de nome
    df_analise_origem.rename(columns={
        'Cliente': 'Clientes',
        'Veiculo': 'Veiculos',
        'UEN': 'UENs',
        'Setor': 'Setores'
    }, inplace=True)

    columns_to_convert = [
        'VL Tabela', 'Vl Bruto', 'Vl Liquido Final', 'IPCA BR', 'IPCA ES',
        'Taxa Ac. TRI % PIB', 'PMC - Número-índice (2022=100) (Número-índice)/ ES',
        'PMC - Número-índice (2022=100) (Número-índice)/ BR',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES',
        'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES',
        'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR'
    ]
    
    for col in columns_to_convert:
        df_analise_origem[col] = pd.to_numeric(df_analise_origem[col], errors='coerce')
        df_analise_origem.dropna(inplace=True)

    DFtoModel = df_analise_origem[df_analise_origem['Origem'] == model].reset_index(drop=True)

    return DFtoModel

In [8]:
database = pd.read_csv('BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.xlsx - BASE.csv')
external_data = pd.read_csv('DadosEconomicos_ES_Inteli.xlsx - VarejoTotal.csv')


In [9]:
cleaned = clean(database, external_data)
df = createDFbyUEN(cleaned, 'Rádio')

C:\Users\Inteli\AppData\Local\Temp\ipykernel_4168\875472363.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
C:\Users\Inteli\AppData\Local\Temp\ipykernel_4168\875472363.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pmc['Data'] = pd.to_datetime(pmc['Mês'].replace(meses_map, regex=True).str.replace(r'(\w+)-(\d+)',
C:\Users\Inteli\AppData\Local\Temp\ipykernel_4168\875472363.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [10]:
df

,Data,VL Tabela,Vl Bruto,Vl Liquido Final,IPCA BR,IPCA ES,Taxa Ac. TRI % PIB,PMC - Número-índice (2022=100) (Número-índice)/ ES,PMC - Número-índice (2022=100) (Número-índice)/ BR,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR,"PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES","PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR"
0,2020-01-31,1697272.95,980612.93,819268.49,0.21,0.29,1.1,84.25384,95.01804,87.06581,97.86726,1.8,-1.9
1,2020-02-29,1869145.18,1197334.21,985286.15,0.25,0.33,1.1,81.61827,90.61413,86.78925,98.12855,-0.3,0.3
2,2020-03-31,1540046.61,878104.69,698138.91,0.07,0.12,1.1,80.76074,90.34239,83.15137,93.91747,-4.2,-4.3
3,2020-04-30,830824.54,517703.10,438250.43,-0.31,-0.09,-1.7,66.43382,75.01668,69.94442,79.16208,-15.9,-15.7
4,2020-05-31,790842.28,448276.65,360395.43,-0.38,-0.48,-1.7,81.57634,86.89620,82.56763,88.37704,18.0,11.6
5,2020-06-30,852452.63,497641.49,418342.40,0.26,0.56,-1.7,83.31604,91.20908,88.39614,95.16073,7.1,7.7
6,2020-07-31,908475.04,517230.62,426543.02,0.36,0.21,-2.8,89.13238,99.06405,88.73469,99.56395,0.4,4.6
7,2020-08-31,1088473.27,604084.86,505510.47,0.24,-0.03,-2.8,90.26367,102.25301,90.99657,102.33392,2.5,2.8
8,2020-09-30,1210216.48,673717.37,559152.74,0.64,0.83,-2.8,90.83188,99.17828,93.48728,103.47595,2.7,1.1
9,2020-10-31,1609171.16,1062086.58,890277.97,0.86,0.91,-3.3,99.50737,105.88980,96.57693,104.42429,3.3,0.9


In [11]:
cleaned2 = clean(database, external_data)
df2 = createDFbyOrigem(cleaned2, 'MP - MÍDIA PROGRAMÁTICA')

C:\Users\Inteli\AppData\Local\Temp\ipykernel_4168\875472363.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
C:\Users\Inteli\AppData\Local\Temp\ipykernel_4168\875472363.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pmc['Data'] = pd.to_datetime(pmc['Mês'].replace(meses_map, regex=True).str.replace(r'(\w+)-(\d+)',
C:\Users\Inteli\AppData\Local\Temp\ipykernel_4168\875472363.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [12]:
df2

,AnoMes,Origem,VL Tabela,Vl Bruto,Vl Liquido Final,Clientes,Veiculos,UENs,Setores,IPCA BR,IPCA ES,Taxa Ac. TRI % PIB,PMC - Número-índice (2022=100) (Número-índice)/ ES,PMC - Número-índice (2022=100) (Número-índice)/ BR,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR,"PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES","PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR",Contratos
0,2020-01,MP - MÍDIA PROGRAMÁTICA,269999.31,269999.31,269999.31,10,2,2,5,0.21,0.29,1.1,84.25384,95.01804,87.06581,97.86726,1.8,-1.9,428
1,2020-02,MP - MÍDIA PROGRAMÁTICA,128804.97,128804.97,128804.97,10,2,2,4,0.25,0.33,1.1,81.61827,90.61413,86.78925,98.12855,-0.3,0.3,428
2,2020-03,MP - MÍDIA PROGRAMÁTICA,178349.82,178349.82,178349.82,9,2,2,5,0.07,0.12,1.1,80.76074,90.34239,83.15137,93.91747,-4.2,-4.3,428
3,2020-04,MP - MÍDIA PROGRAMÁTICA,158171.01,158171.01,158171.01,11,2,2,5,-0.31,-0.09,-1.7,66.43382,75.01668,69.94442,79.16208,-15.9,-15.7,148
4,2020-05,MP - MÍDIA PROGRAMÁTICA,118444.19,118444.19,118444.19,10,2,2,4,-0.38,-0.48,-1.7,81.57634,86.89620,82.56763,88.37704,18.0,11.6,428
5,2020-06,MP - MÍDIA PROGRAMÁTICA,143352.53,143352.53,143352.53,11,2,2,5,0.26,0.56,-1.7,83.31604,91.20908,88.39614,95.16073,7.1,7.7,428
6,2020-07,MP - MÍDIA PROGRAMÁTICA,117187.30,117187.30,117187.30,11,2,2,6,0.36,0.21,-2.8,89.13238,99.06405,88.73469,99.56395,0.4,4.6,428
7,2020-08,MP - MÍDIA PROGRAMÁTICA,120644.84,120644.84,120644.84,10,2,2,5,0.24,-0.03,-2.8,90.26367,102.25301,90.99657,102.33392,2.5,2.8,111
8,2020-09,MP - MÍDIA PROGRAMÁTICA,161031.26,161031.26,161031.26,10,2,2,5,0.64,0.83,-2.8,90.83188,99.17828,93.48728,103.47595,2.7,1.1,428
9,2020-10,MP - MÍDIA PROGRAMÁTICA,188123.21,188123.21,188123.21,11,2,2,5,0.86,0.91,-3.3,99.50737,105.88980,96.57693,104.42429,3.3,0.9,428


In [13]:
import pandas as pd

df = pd.read_csv('data/results/dataframes/DFtoModel.csv')

df

,VL Tabela,Vl Bruto,Vl Liquido Final,Clientes,Veiculos,UENs,Setores,IPCA BR,IPCA ES,Taxa Ac. TRI % PIB,PMC - Número-índice (2022=100) (Número-índice)/ ES,PMC - Número-índice (2022=100) (Número-índice)/ BR,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR,"PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES","PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR",Contratos
0,1021245.74,740082.18,631609.24,84,8,3,13,0.21,0.29,1.1,84.25384,95.01804,87.06581,97.86726,1.8,-1.9,428
1,858887.02,670338.90,571058.87,84,10,3,14,0.25,0.33,1.1,81.61827,90.61413,86.78925,98.12855,-0.3,0.3,428
2,740737.25,584429.99,494912.06,88,9,3,16,0.07,0.12,1.1,80.76074,90.34239,83.15137,93.91747,-4.2,-4.3,428
3,580090.56,441628.97,367961.98,90,10,3,13,-0.31,-0.09,-1.7,66.43382,75.01668,69.94442,79.16208,-15.9,-15.7,428
4,739874.85,545292.44,454648.49,64,7,3,13,-0.38,-0.48,-1.7,81.57634,86.89620,82.56763,88.37704,18.0,11.6,428
5,742898.26,524300.30,449375.03,81,10,3,14,0.26,0.56,-1.7,83.31604,91.20908,88.39614,95.16073,7.1,7.7,428
6,668588.12,489133.94,404561.43,72,8,3,15,0.36,0.21,-2.8,89.13238,99.06405,88.73469,99.56395,0.4,4.6,428
7,638674.31,474671.99,401328.14,84,9,3,14,0.24,-0.03,-2.8,90.26367,102.25301,90.99657,102.33392,2.5,2.8,428
8,910373.40,608416.77,516580.49,106,8,3,17,0.64,0.83,-2.8,90.83188,99.17828,93.48728,103.47595,2.7,1.1,428
9,1123070.40,632802.49,528778.81,103,8,3,17,0.86,0.91,-3.3,99.50737,105.88980,96.57693,104.42429,3.3,0.9,428


In [14]:
import pandas as pd

df = pd.read_csv('data/results/dataframes/DFtoModel.csv')

df

,VL Tabela,Vl Bruto,Vl Liquido Final,Clientes,Veiculos,UENs,Setores,IPCA BR,IPCA ES,Taxa Ac. TRI % PIB,PMC - Número-índice (2022=100) (Número-índice)/ ES,PMC - Número-índice (2022=100) (Número-índice)/ BR,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES,PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR,"PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES","PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR",Contratos
0,1021245.74,740082.18,631609.24,84,8,3,13,0.21,0.29,1.1,84.25384,95.01804,87.06581,97.86726,1.8,-1.9,428
1,858887.02,670338.90,571058.87,84,10,3,14,0.25,0.33,1.1,81.61827,90.61413,86.78925,98.12855,-0.3,0.3,428
2,740737.25,584429.99,494912.06,88,9,3,16,0.07,0.12,1.1,80.76074,90.34239,83.15137,93.91747,-4.2,-4.3,428
3,580090.56,441628.97,367961.98,90,10,3,13,-0.31,-0.09,-1.7,66.43382,75.01668,69.94442,79.16208,-15.9,-15.7,428
4,739874.85,545292.44,454648.49,64,7,3,13,-0.38,-0.48,-1.7,81.57634,86.89620,82.56763,88.37704,18.0,11.6,428
5,742898.26,524300.30,449375.03,81,10,3,14,0.26,0.56,-1.7,83.31604,91.20908,88.39614,95.16073,7.1,7.7,428
6,668588.12,489133.94,404561.43,72,8,3,15,0.36,0.21,-2.8,89.13238,99.06405,88.73469,99.56395,0.4,4.6,428
7,638674.31,474671.99,401328.14,84,9,3,14,0.24,-0.03,-2.8,90.26367,102.25301,90.99657,102.33392,2.5,2.8,428
8,910373.40,608416.77,516580.49,106,8,3,17,0.64,0.83,-2.8,90.83188,99.17828,93.48728,103.47595,2.7,1.1,428
9,1123070.40,632802.49,528778.81,103,8,3,17,0.86,0.91,-3.3,99.50737,105.88980,96.57693,104.42429,3.3,0.9,428
